<a href="https://colab.research.google.com/github/djliden/numerai/blob/main/run_from_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run from Colab

NB Goal: Set up a system for cloning this package and running the model(s) on colab in a fairly hands-off way.

To authenticate, either upload your `.env` file with the necessary API keys to the numerai folder which you will clone below, or you will be prompted to copy the keys.

In [1]:
!git clone https://github.com/djliden/numerai.git
%cd numerai

Cloning into 'numerai'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 180 (delta 90), reused 128 (delta 52), pack-reused 0
Receiving objects: 100% (180/180), 139.19 KiB | 874.00 KiB/s, done.
Resolving deltas: 100% (90/90), done.
/content/numerai


In [3]:
%%capture
%%bash
pip install --upgrade fastai numerapi python-dotenv ipywidgets jupyter ipython ipython-genutils yacs

In [ ]:
!python ./src/fastai_tabular.py ./src/config/experiments/expt_colab.yaml

DATA:
  REFRESH: False
  SAVE_PROCESSED_TRAIN: True
EVAL:
  SAVE_PREDS: False
MODEL:
  BATCHNORM_FINAL: False
  DROPOUT_P: 0
  EMBED_DROPOUT_P: 0
  GROUP: fastai_tabular
  LAYERS: [500, 300]
  NAME: size_500_300
  USE_BATCHNORM: False
  WEIGHT_DECAY: 0
  Y_RANGE: [0, 0]
RESULTS:
  
SESSION:
  NAME: Default
SYSTEM:
  DEBUG: False
  TIME: 2021_2_28_111
TRAIN:
  N_EPOCHS: 6
Loaded Numerai Public Key into Global Environment!
Loaded Numerai Secret Key into Global Environment!
input/numerai_dataset_253.zip: 100% 392M/393M [00:25<00:00, 17.4MB/s]2021-02-28 01:12:12,228 INFO numerapi.base_api: unzipping file...


## Outstanding Issues:
- Progress bars not rendering correctly.

In [ ]:
#!git pull